**MCMC(j)- TCI(k,l)**

In [ ]:
import TensorCrossInterpolation as TCI




function f(v)
    j = v[1]
    k = v[2]
    l = v[3]

    
    term1 = cos(2*π*(j*k/(N*N2))) * sin(2*π*(j*l/(N*N2))) * exp(-j/N)
    term2 = sin(2*π*(j*k/(N*N2))) * cos(2*π*(j*l/(N*N2))) * exp(-j^2/(N^2))

    return term1 + term2
end


function get_tci_val(j)
    localdims = fill(N2, 2)
    tolerance = 1e-8

    g(v) = f([j, v[1], v[2]]) #the method of fix one variable
    
    tci_2d, _, _ = TCI.crossinterpolate2(Float64, g, localdims; tolerance = tolerance )
    
    mps = tci_2d.sitetensors
    
    sumK = dropdims(sum(mps[1],dims=2),dims=2) #(r1 X N X r2) -> (r1 X 1 X r2) -> (r1 x r2)
    sumL = dropdims(sum(mps[2],dims=2),dims=2)

    resultMatrix = sumK * sumL #1x1 matrix
    
    return resultMatrix[1]

end


function mcmc_tci_sum(M)
    #define sum value
    sumVal = 0.0

    #set init point
    j = rand(1:N)
    now = get_tci_val(j)
    memJ[j] = now
    sumVal += now
    

    for idx in 1:(M-1)
        jump = rand(-round(Int,N/100):round(Int,N/100))
        
        if (j + jump) < 1 || (j + jump) > N
            sumVal += now
            continue
        else
            j = j + jump

            #calculate proposal point
            prp = 0.0
            if memJ[j] == 0
                prp = get_tci_val(j)
                memJ[j] = prp
            else
                prp = memJ[j]
            end

            #MCMC
            accRatio = abs(prp/now)
            if rand() < accRatio
                #acc
                sumVal += sign(prp)
                now = prp
            else
                #rej
                sumVal += now 
                j = j - jump #undo
            end
        end
    end

    return sumVal
end

function exact_sum()
    sum = 0.0
    
    for j in 1:N
        for k in 1:N2
            for l in 1:N2
                sum += f([j, k, l])
            end
        end
    end
    return sum / (N*(N2^2))
end

N = 100 #Num of j elements
N2 = 100 #Num of k,l elements
MLst = [36_000] #number of sampling
memJ = zeros(N)


extVal = exact_sum()
println("exact result : $extVal")

 for M in MLst
    println("====M : $M====")
    epoch = 10
    acc = 0
    accSq = 0
    for i in 1:epoch
        mcmcTciSum = mcmc_tci_sum(M) / ((N2^2)*M)
        println(mcmcTciSum)
        acc += mcmcTciSum
        accSq += mcmcTciSum^2
    end

    errVal = sqrt((accSq / epoch - (acc / epoch)^2)/epoch)

    println("MCMC-TCI result : $(acc/epoch)")
    println("errVal/sqrt(M) : $errVal")

    if extVal - (errVal/sqrt(M)) <  (acc/epoch) && (acc/epoch) < extVal + (errVal/sqrt(M))
        println("Success :)")
    else
        println("Fail :(")
    end

end

exact result : 0.1956521400764422
====M : 36000====
-7.0383197359894956e-6
-7.267416877711919e-6
-7.260636284369451e-6
-7.302698479814684e-6
-6.866225075175928e-6
-7.329854968606739e-6
5.6419843314882895e-5
5.2859961033110307e-5
5.725272598197908e-5
5.694315518683676e-5
MCMC-TCI result : 1.8041053409514084e-5
errVal/sqrt(M) : 9.77355975661554e-6
Fail :(


**Proposal Distribution**
https://jduarte.physics.ucsd.edu/phys142/lectures/16_Path_Integral_MCMC.pdf (page 9)\
There's no need to use complicated proposal distribution. \
We can simply use vuniform distribution.